<a href="https://colab.research.google.com/github/Durgesh-18/iitd-tryst-mule-accounts-eda/blob/main/Mule_Account_Detection_EDA_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# 1. LOAD DATA


In [ ]:

customers = pd.read_csv("customers.csv")
accounts = pd.read_csv("accounts.csv")
linkage = pd.read_csv("customer_account_linkage.csv")
products = pd.read_csv("product_details.csv")
labels = pd.read_csv("train_labels.csv")
test = pd.read_csv("test_accounts.csv")

print("Loading 6 transaction files...")
transactions = pd.concat(
    [pd.read_csv(f"transactions_part_{i}.csv") for i in range(6)],
    ignore_index=True
)

print("\n--- Shapes ---")
for name, df in [("customers", customers), ("accounts", accounts),
                 ("linkage", linkage), ("products", products),
                 ("labels", labels), ("test", test),
                 ("transactions", transactions)]:
    print(f"  {name}: {df.shape}")

Loading 6 transaction files...

--- Shapes ---
  customers: (39988, 14)
  accounts: (40038, 22)
  linkage: (40038, 2)
  products: (39988, 11)
  labels: (24023, 5)
  test: (16015, 1)
  transactions: (1044796, 8)


# 2. BASIC INSPECTION

In [ ]:

print("\n" + "=" * 60)
print("DTYPES & MISSING VALUES")
print("=" * 60)

for name, df in [("customers", customers), ("accounts", accounts),
                 ("linkage", linkage), ("products", products),
                 ("labels", labels), ("transactions", transactions)]:
    print(f"\n--- {name} ---")
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    print(f"  Shape: {df.shape}")
    print(f"  Dtypes:\n{df.dtypes.to_string()}")
    if len(missing):
        print(f"  Missing:\n{missing.to_string()}")
    else:
        print("  No missing values")


DTYPES & MISSING VALUES

--- customers ---
  Shape: (39988, 14)
  Dtypes:
customer_id                object
date_of_birth              object
relationship_start_date    object
pan_available              object
aadhaar_available          object
passport_available         object
mobile_banking_flag        object
internet_banking_flag      object
atm_card_flag              object
demat_flag                 object
credit_card_flag           object
fastag_flag                object
customer_pin                int64
permanent_pin               int64
  Missing:
pan_available        5732
aadhaar_available    9708

--- accounts ---
  Shape: (40038, 22)
  Dtypes:
account_id                  object
account_status              object
product_code                 int64
currency_code                int64
account_opening_date        object
branch_code                  int64
branch_pin                 float64
avg_balance                float64
product_family              object
nomination_flag       


# 3. TARGET VARIABLE

In [ ]:

print("\n" + "=" * 60)
print("TARGET VARIABLE")
print("=" * 60)

print(labels["is_mule"].value_counts())
print(f"Mule rate: {labels['is_mule'].mean():.4f} ({labels['is_mule'].mean()*100:.2f}%)")

# Alert reasons
print("\n--- Alert Reasons ---")
print(labels[labels.is_mule == 1]["alert_reason"].value_counts(dropna=False))

# Mule flag date distribution
labels["mule_flag_date"] = pd.to_datetime(labels["mule_flag_date"])
print("\n--- Mule Flag Year Distribution ---")
print(labels[labels.is_mule == 1]["mule_flag_date"].dt.year.value_counts().sort_index())


TARGET VARIABLE
is_mule
0    23760
1      263
Name: count, dtype: int64
Mule rate: 0.0109 (1.09%)

--- Alert Reasons ---
alert_reason
Routine Investigation                       55
Rapid Movement of Funds                     22
NaN                                         21
Structuring Transactions Below Threshold    18
Branch Cluster Investigation                17
Dormant Account Reactivation                17
Unusual Fund Flow Pattern                   17
Income-Transaction Mismatch                 17
High-Value Activity on New Account          16
Post-Contact-Update Spike                   14
Geographic Anomaly Detected                 13
Layered Transaction Pattern                 12
Round Amount Pattern                        12
Salary Cycle Anomaly                        12
Name: count, dtype: int64

--- Mule Flag Year Distribution ---
mule_flag_date
2017      1
2018      1
2020      3
2022     18
2023     24
2024     46
2025    159
2026     11
Name: count, dtype: int64


#BUILDING MASTER TRAIN TABLE

In [ ]:
# Parse dates
for df, cols in [
    (accounts, ["account_opening_date", "last_mobile_update_date",
                "last_kyc_date", "freeze_date", "unfreeze_date"]),
    (customers, ["date_of_birth", "relationship_start_date"]),
    (labels,    ["mule_flag_date"])
]:
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")

train = (labels
         .merge(accounts, on="account_id", how="left")
         .merge(linkage, on="account_id", how="left")
         .merge(customers, on="customer_id", how="left")
         .merge(products, on="customer_id", how="left"))

REF_DATE = pd.Timestamp("2025-06-30")

train["age"]                    = (REF_DATE - train["date_of_birth"]).dt.days / 365.25
train["relationship_tenure_days"] = (REF_DATE - train["relationship_start_date"]).dt.days
train["account_age_days"]       = (REF_DATE - train["account_opening_date"]).dt.days
train["is_frozen"]              = train["freeze_date"].notna().astype(int)
train["has_mobile_update"]      = train["last_mobile_update_date"].notna().astype(int)
train["opening_year"]           = train["account_opening_date"].dt.year
train["pin_match"]              = (train["customer_pin"] == train["permanent_pin"]).astype(int)

# Accounts per customer
accts_per_cust = linkage.groupby("customer_id")["account_id"].count().rename("num_accounts").reset_index()
train = train.merge(accts_per_cust, on="customer_id", how="left")

print(f"Master train table shape: {train.shape}")


BUILDING MASTER TRAIN TABLE
Master train table shape: (24023, 58)


# ACCOUNT-LEVEL ANALYSIS

In [ ]:

cat_cols = ["account_status", "product_family", "kyc_compliant",
            "nomination_flag", "rural_branch", "cheque_availed",
            "mobile_banking_flag", "internet_banking_flag",
            "atm_card_flag", "credit_card_flag", "demat_flag",
            "is_frozen", "has_mobile_update", "pin_match"]

for col in cat_cols:
    if col in train.columns:
        ct = pd.crosstab(train[col], train["is_mule"], normalize="columns").round(3)
        print(f"\n--- {col} ---")
        print(ct)

print("\n--- Account Opening Year (column-normalized) ---")
print(pd.crosstab(train["opening_year"], train["is_mule"], normalize="columns").round(3))


ACCOUNT-LEVEL ANALYSIS

--- account_status ---
is_mule            0      1
account_status             
active          0.98  0.601
frozen          0.02  0.399

--- product_family ---
is_mule             0      1
product_family              
K               0.318  0.331
O               0.100  0.103
S               0.582  0.567

--- kyc_compliant ---
is_mule          0      1
kyc_compliant            
N              0.1  0.084
Y              0.9  0.916

--- nomination_flag ---
is_mule              0      1
nomination_flag              
N                0.396  0.411
Y                0.604  0.589

--- rural_branch ---
is_mule           0     1
rural_branch             
N             0.883  0.84
Y             0.117  0.16

--- cheque_availed ---
is_mule             0      1
cheque_availed              
N               0.638  0.601
Y               0.362  0.399

--- mobile_banking_flag ---
is_mule                 0      1
mobile_banking_flag             
N                    0.68  0.662
Y    

# ACCOUNT-LEVEL ANALYSIS

In [ ]:
balance_cols = ["avg_balance", "monthly_avg_balance",
                "quarterly_avg_balance", "daily_avg_balance"]

for col in balance_cols:
    if col in train.columns:
        print(f"\n--- {col} ---")
        print(train.groupby("is_mule")[col].describe().round(0))

train["balance_spread"] = train["daily_avg_balance"] - train["quarterly_avg_balance"]
print("\n--- Balance Spread (daily - quarterly) ---")
print(train.groupby("is_mule")["balance_spread"].describe().round(0))

train["has_negative_balance"] = (train["avg_balance"] < 0).astype(int)
print("\n--- Negative Balance Rate ---")
print(pd.crosstab(train["has_negative_balance"], train["is_mule"], normalize="columns").round(3))



BALANCE ANALYSIS

--- avg_balance ---
           count     mean       std         min    25%     50%      75%         max
is_mule                                                                            
0        23044.0  53282.0  859812.0 -19251711.0  518.0  5260.0  27244.0  54397082.0
1          254.0 -26562.0  567915.0  -8085936.0  357.0  3561.0  18122.0   1101870.0

--- monthly_avg_balance ---
           count     mean       std         min    25%     50%      75%         max
is_mule                                                                            
0        23044.0  52861.0  862957.0 -18004782.0  506.0  5214.0  27221.0  54182818.0
1          254.0 -20981.0  493839.0  -6664511.0  343.0  3394.0  19061.0   1042998.0

--- quarterly_avg_balance ---
           count     mean       std         min    25%     50%      75%         max
is_mule                                                                            
0        23760.0  51438.0  878389.0 -23407343.0  496.0  5130.

# DEMOGRAPHIC & TEMPORAL ANALYSIS

In [ ]:

for col in ["age", "relationship_tenure_days", "account_age_days", "num_accounts"]:
    print(f"\n--- {col} ---")
    print(train.groupby("is_mule")[col].describe().round(1))

# Product counts
for col in ["loan_count", "cc_count", "od_count", "ka_count", "sa_count"]:
    if col in train.columns:
        m = train[train.is_mule == 1][col].mean()
        l = train[train.is_mule == 0][col].mean()
        print(f"{col}: mule_mean={m:.3f}, legit_mean={l:.3f}")

print("\n--- SA Count Distribution ---")
print(pd.crosstab(train["sa_count"], train["is_mule"], normalize="columns").round(3))

# Mobile update timing
mules = train[train.is_mule == 1].copy()
mules["days_mobile_to_flag"] = (mules["mule_flag_date"] - mules["last_mobile_update_date"]).dt.days
has_mobile = mules["days_mobile_to_flag"].dropna()
print(f"\n--- Mobile Update to Mule Flag (days) ---")
print(has_mobile.describe().round(0))
print(f"Within 90 days: {(has_mobile < 90).sum()} / {len(has_mobile)}")
print(f"Within 30 days: {(has_mobile < 30).sum()} / {len(has_mobile)}")

# Days from account open to flag
mules["days_open_to_flag"] = (mules["mule_flag_date"] - mules["account_opening_date"]).dt.days
print("\n--- Days from Account Open to Mule Flag ---")
print(mules["days_open_to_flag"].describe().round(0))
neg = (mules["days_open_to_flag"] < 0).sum()
print(f"Negative (flag before open — data error): {neg}")


DEMOGRAPHIC & TEMPORAL ANALYSIS

--- age ---
           count  mean   std   min   25%   50%   75%   max
is_mule                                                   
0        23760.0  49.5  17.8  18.5  34.4  49.5  64.9  80.5
1          263.0  50.9  18.6  18.9  36.2  49.8  68.7  80.1

--- relationship_tenure_days ---
           count    mean     std   min     25%     50%     75%      max
is_mule                                                                
0        23760.0  5622.1  3216.2  30.0  2855.0  5634.5  8409.2  11138.0
1          263.0  5704.7  3224.8  33.0  3017.0  5671.0  8558.0  11105.0

--- account_age_days ---
           count    mean    std   min    25%    50%     75%     max
is_mule                                                            
0        23760.0  1124.4  820.4  23.0  639.0  805.0  1330.0  3650.0
1          263.0   989.3  773.3  10.0  570.5  751.0  1168.5  3544.0

--- num_accounts ---
           count  mean  std  min  25%  50%  75%  max
is_mule                

# BRANCH ANALYSIS

In [ ]:
branch_stats = (train.groupby("branch_code")
                .agg(total=("is_mule", "count"), mules=("is_mule", "sum"))
                .assign(mule_rate=lambda x: x["mules"] / x["total"])
                .reset_index())

for min_accts in [5, 10, 20]:
    filtered = branch_stats[branch_stats["total"] >= min_accts]
    print(f"\nBranches with >= {min_accts} labeled accounts: {len(filtered)}")
    print(f"  mule_rate > 5%:  {(filtered['mule_rate'] > 0.05).sum()}")
    print(f"  mule_rate > 10%: {(filtered['mule_rate'] > 0.10).sum()}")

print("\nTop 15 branches by mule_rate (>= 5 accounts):")
print(branch_stats[branch_stats["total"] >= 5]
      .sort_values("mule_rate", ascending=False)
      .head(15)
      .to_string(index=False))



BRANCH ANALYSIS

Branches with >= 5 labeled accounts: 1196
  mule_rate > 5%:  79
  mule_rate > 10%: 78

Branches with >= 10 labeled accounts: 12
  mule_rate > 5%:  1
  mule_rate > 10%: 0

Branches with >= 20 labeled accounts: 0
  mule_rate > 5%:  0
  mule_rate > 10%: 0

Top 15 branches by mule_rate (>= 5 accounts):
 branch_code  total  mules  mule_rate
        4091      7      6   0.857143
        8103      7      4   0.571429
        2390      5      2   0.400000
        1936      5      1   0.200000
        4129      5      1   0.200000
        7455      5      1   0.200000
        4119      5      1   0.200000
        4644      5      1   0.200000
        9061      5      1   0.200000
        5243      5      1   0.200000
        5229      5      1   0.200000
        3512      5      1   0.200000
        1320      5      1   0.200000
        4816      5      1   0.200000
        6786      5      1   0.200000


# FREEZE DATE ANALYSIS

In [ ]:
all_acc = accounts.merge(labels, on="account_id", how="left")
frozen = all_acc[all_acc["freeze_date"].notna()]
print(f"Total frozen accounts: {len(frozen)}")
print(f"  Frozen + mule:     {(frozen['is_mule'] == 1).sum()}")
print(f"  Frozen + legit:    {(frozen['is_mule'] == 0).sum()}")
print(f"  Frozen + unlabeled:{frozen['is_mule'].isna().sum()}")

mules_frozen = mules[mules["freeze_date"].notna()].copy()
mules_frozen["days_flag_to_freeze"] = (mules_frozen["freeze_date"] - mules_frozen["mule_flag_date"]).dt.days
print("\n--- Days from Mule Flag to Freeze (mules only) ---")
print(mules_frozen["days_flag_to_freeze"].describe().round(0))
print(f"Freeze BEFORE flag (independent risk signal): {(mules_frozen['days_flag_to_freeze'] < 0).sum()}")


FREEZE DATE ANALYSIS
Total frozen accounts: 1317
  Frozen + mule:     155
  Frozen + legit:    708
  Frozen + unlabeled:454

--- Days from Mule Flag to Freeze (mules only) ---
count     155.0
mean      103.0
std       474.0
min      -943.0
25%      -200.0
50%        -2.0
75%       186.0
max      2608.0
Name: days_flag_to_freeze, dtype: float64
Freeze BEFORE flag (independent risk signal): 79


# TRANSACTION ANALYSIS

In [ ]:
transactions["transaction_timestamp"] = pd.to_datetime(
    transactions["transaction_timestamp"], errors="coerce"
)

print(f"Total transactions: {len(transactions):,}")
print(f"Date range: {transactions['transaction_timestamp'].min()} to {transactions['transaction_timestamp'].max()}")
print(f"Unique accounts: {transactions['account_id'].nunique():,}")
print(f"Unique counterparties: {transactions['counterparty_id'].nunique():,}")

print("\n--- Transaction Types ---")
print(transactions["txn_type"].value_counts())

print("\n--- Channel Distribution ---")
print(transactions["channel"].value_counts().head(20))

print("\n--- Amount Distribution (all transactions) ---")
print(transactions["amount"].describe().round(0))

# Negative amounts (reversals)
rev = (transactions["amount"] < 0).sum()
print(f"\nNegative amount (reversals): {rev:,} ({rev/len(transactions)*100:.2f}%)")


TRANSACTION ANALYSIS
Total transactions: 1,044,796
Date range: 2020-07-01 00:10:03 to 2025-04-02 11:41:13
Unique accounts: 32,990
Unique counterparties: 92,447

--- Transaction Types ---
txn_type
D    563297
C    481495
Name: count, dtype: int64

--- Channel Distribution ---
channel
UPC    399086
UPD    372458
IPM     43403
P2A     29285
END     25033
NTD     21053
FTD     14431
STD     10903
MCR     10493
MAC     10395
RCD     10339
FTC      9301
TPD      8341
CHQ      7635
APD      6767
ATW      6214
ETD      5867
TPC      5622
NWD      5407
STC      5074
Name: count, dtype: int64

--- Amount Distribution (all transactions) ---
count     1044792.0
mean         9675.0
std         59412.0
min      -4700500.0
25%           137.0
50%           900.0
75%          4483.0
max      12572000.0
Name: amount, dtype: float64

Negative amount (reversals): 5,172 (0.50%)


# PER-ACCOUNT TRANSACTION AGGREGATES

In [ ]:
credits = transactions[transactions["txn_type"] == "C"]
debits  = transactions[transactions["txn_type"] == "D"]

credit_agg = credits.groupby("account_id").agg(
    total_credit_amount=("amount", "sum"),
    credit_count=("amount", "count"),
    avg_credit_amount=("amount", "mean"),
    max_credit_amount=("amount", "max"),
    unique_credit_counterparties=("counterparty_id", "nunique")
).reset_index()

debit_agg = debits.groupby("account_id").agg(
    total_debit_amount=("amount", "sum"),
    debit_count=("amount", "count"),
    avg_debit_amount=("amount", "mean"),
    max_debit_amount=("amount", "max"),
    unique_debit_counterparties=("counterparty_id", "nunique")
).reset_index()

txn_agg = (transactions.groupby("account_id").agg(
    total_txn_count=("transaction_id", "count"),
    unique_counterparties=("counterparty_id", "nunique"),
    unique_channels=("channel", "nunique"),
    first_txn=("transaction_timestamp", "min"),
    last_txn=("transaction_timestamp", "max")
).reset_index()
.merge(credit_agg, on="account_id", how="left")
.merge(debit_agg,  on="account_id", how="left"))

txn_agg["net_flow"] = txn_agg["total_credit_amount"].fillna(0) + txn_agg["total_debit_amount"].fillna(0)
txn_agg["pass_through_ratio"] = (
    txn_agg["net_flow"].abs() /
    (txn_agg["total_credit_amount"].fillna(0).abs() + txn_agg["total_debit_amount"].fillna(0).abs() + 1e-9)
)
txn_agg["credit_debit_ratio"] = txn_agg["credit_count"] / (txn_agg["debit_count"] + 1e-9)

# Merge with labels
txn_labeled = txn_agg.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")

print("\n--- Total Transaction Count ---")
print(txn_labeled.groupby("is_mule")["total_txn_count"].describe().round(0))

print("\n--- Net Flow (credit + debit, debit is negative) ---")
print(txn_labeled.groupby("is_mule")["net_flow"].describe().round(0))

print("\n--- Pass-Through Ratio (0=perfect pass-through, 1=one-directional) ---")
print(txn_labeled.groupby("is_mule")["pass_through_ratio"].describe().round(3))

print("\n--- Unique Counterparties ---")
print(txn_labeled.groupby("is_mule")["unique_counterparties"].describe().round(0))

print("\n--- Total Credit Amount ---")
print(txn_labeled.groupby("is_mule")["total_credit_amount"].describe().round(0))

print("\n--- Max Single Credit ---")
print(txn_labeled.groupby("is_mule")["max_credit_amount"].describe().round(0))

print("\n--- Credit/Debit Count Ratio ---")
print(txn_labeled.groupby("is_mule")["credit_debit_ratio"].describe().round(3))


PER-ACCOUNT TRANSACTION AGGREGATES

--- Total Transaction Count ---
           count  mean   std  min  25%   50%   75%    max
is_mule                                                  
0        19571.0  32.0  83.0  1.0  3.0   8.0  24.0  925.0
1          227.0  32.0  67.0  1.0  4.0  10.0  24.0  593.0

--- Net Flow (credit + debit, debit is negative) ---
           count      mean       std        min      25%       50%       75%         max
is_mule                                                                                 
0        19571.0  309100.0  820911.0 -1094671.0   6000.0   47578.0  243600.0  11302783.0
1          227.0  448625.0  716836.0        8.0  49432.0  187926.0  559049.0   5054124.0

--- Pass-Through Ratio (0=perfect pass-through, 1=one-directional) ---
           count   mean    std    min  25%  50%  75%  max
is_mule                                                  
0        19571.0  0.999  0.028  0.032  1.0  1.0  1.0  1.0
1          227.0  1.000  0.000  1.000  1.0 

# PATTERN-SPECIFIC ANALYSIS

In [ ]:

# --- Structuring: amounts just below 50k or 100k ---
transactions["near_50k"] = transactions["amount"].between(45000, 50000)
transactions["near_100k"] = transactions["amount"].between(90000, 100000)
struct_agg = transactions.groupby("account_id").agg(
    total_txns=("amount", "count"),
    near_50k_count=("near_50k", "sum"),
    near_100k_count=("near_100k", "sum")
).reset_index()
struct_agg["structuring_ratio"] = (struct_agg["near_50k_count"] + struct_agg["near_100k_count"]) / struct_agg["total_txns"]
struct_labeled = struct_agg.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")
print("\n--- Structuring Ratio (txns near 50k/100k) ---")
print(struct_labeled.groupby("is_mule")["structuring_ratio"].describe().round(4))

# --- Round amounts ---
transactions["is_round"] = (transactions["amount"].abs() % 1000 == 0)
round_agg = transactions.groupby("account_id").agg(
    total=("amount", "count"),
    round_count=("is_round", "sum")
).reset_index()
round_agg["round_ratio"] = round_agg["round_count"] / round_agg["total"]
round_labeled = round_agg.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")
print("\n--- Round Amount Ratio ---")
print(round_labeled.groupby("is_mule")["round_ratio"].describe().round(3))

# --- Time of day ---
transactions["hour"] = transactions["transaction_timestamp"].dt.hour
transactions["is_night"] = transactions["hour"].between(23, 5) | (transactions["hour"] < 5)
night_agg = transactions.groupby("account_id").agg(
    total=("amount", "count"),
    night_count=("is_night", "sum")
).reset_index()
night_agg["night_ratio"] = night_agg["night_count"] / night_agg["total"]
night_labeled = night_agg.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")
print("\n--- Night Transaction Ratio (11pm-5am) ---")
print(night_labeled.groupby("is_mule")["night_ratio"].describe().round(3))

# --- Month boundary transactions ---
transactions["day_of_month"] = transactions["transaction_timestamp"].dt.day
transactions["month"] = transactions["transaction_timestamp"].dt.days_in_month
transactions["is_month_boundary"] = (
    (transactions["day_of_month"] <= 3) |
    (transactions["day_of_month"] >= transactions["transaction_timestamp"].dt.days_in_month - 2)
)
boundary_agg = transactions.groupby("account_id").agg(
    total=("amount", "count"),
    boundary_count=("is_month_boundary", "sum")
).reset_index()
boundary_agg["boundary_ratio"] = boundary_agg["boundary_count"] / boundary_agg["total"]
boundary_labeled = boundary_agg.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")
print("\n--- Month Boundary Transaction Ratio ---")
print(boundary_labeled.groupby("is_mule")["boundary_ratio"].describe().round(3))

# --- Channel analysis ---
channel_dummies = pd.get_dummies(transactions["channel"], prefix="ch")
channel_df = pd.concat([transactions[["account_id"]], channel_dummies], axis=1)
channel_agg = channel_df.groupby("account_id").mean().reset_index()
channel_labeled = channel_agg.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")

print("\n--- Channel Usage: Mule vs Legitimate (mean fraction) ---")
channel_cols = [c for c in channel_labeled.columns if c.startswith("ch_")]
channel_compare = channel_labeled.groupby("is_mule")[channel_cols].mean().T
channel_compare.columns = ["legit", "mule"]
channel_compare["diff"] = channel_compare["mule"] - channel_compare["legit"]
channel_compare["ratio"] = (channel_compare["mule"] + 1e-9) / (channel_compare["legit"] + 1e-9)
print(channel_compare.sort_values("diff", ascending=False).round(4).to_string())



PATTERN-SPECIFIC ANALYSIS

--- Structuring Ratio (txns near 50k/100k) ---
           count    mean     std  min  25%  50%     75%  max
is_mule                                                     
0        19571.0  0.0152  0.0714  0.0  0.0  0.0  0.0000  1.0
1          227.0  0.0807  0.2010  0.0  0.0  0.0  0.0385  1.0

--- Round Amount Ratio ---
           count   mean    std  min  25%    50%    75%  max
is_mule                                                    
0        19571.0  0.173  0.201  0.0  0.0  0.147  0.233  1.0
1          227.0  0.143  0.200  0.0  0.0  0.111  0.186  1.0

--- Night Transaction Ratio (11pm-5am) ---
           count   mean    std  min  25%  50%    75%  max
is_mule                                                  
0        19571.0  0.037  0.102  0.0  0.0  0.0  0.040  1.0
1          227.0  0.045  0.110  0.0  0.0  0.0  0.057  1.0

--- Month Boundary Transaction Ratio ---
           count   mean    std  min    25%    50%    75%  max
is_mule                          

# DORMANCY BURST DETECTION

In [ ]:
# Per account, computing max gap between consecutive transactions
# (Only on labeled accounts to keep it manageable)
labeled_account_ids = labels["account_id"].tolist()
txn_labeled_only = transactions[transactions["account_id"].isin(labeled_account_ids)].copy()
txn_labeled_only = txn_labeled_only.sort_values(["account_id", "transaction_timestamp"])

txn_labeled_only["prev_ts"] = txn_labeled_only.groupby("account_id")["transaction_timestamp"].shift(1)
txn_labeled_only["gap_days"] = (
    txn_labeled_only["transaction_timestamp"] - txn_labeled_only["prev_ts"]
).dt.days

gap_agg = txn_labeled_only.groupby("account_id")["gap_days"].agg(
    max_gap="max",
    mean_gap="mean",
    p90_gap=lambda x: x.quantile(0.9)
).reset_index()

gap_labeled = gap_agg.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")
print("\n--- Max Gap Between Consecutive Transactions (days) ---")
print(gap_labeled.groupby("is_mule")["max_gap"].describe().round(0))

# Dormancy: max_gap > 180 days
gap_labeled["is_dormant"] = (gap_labeled["max_gap"] > 180).astype(int)
print("\n--- Dormancy (max gap > 180 days) ---")
print(pd.crosstab(gap_labeled["is_dormant"], gap_labeled["is_mule"], normalize="columns").round(3))



DORMANCY BURST DETECTION

--- Max Gap Between Consecutive Transactions (days) ---
           count   mean    std  min    25%    50%    75%     max
is_mule                                                         
0        16698.0  362.0  336.0  0.0  145.0  219.0  442.0  1727.0
1          209.0  341.0  314.0  0.0  144.0  219.0  389.0  1476.0

--- Dormancy (max gap > 180 days) ---
is_mule         0      1
is_dormant              
0           0.498  0.467
1           0.502  0.533


# RAPID PASS-THROUGH ANALYSIS

In [ ]:
# For each account: time between a credit and the next debit
txn_sorted = txn_labeled_only.sort_values(["account_id", "transaction_timestamp"])
txn_sorted["next_type"] = txn_sorted.groupby("account_id")["txn_type"].shift(-1)
txn_sorted["next_ts"]   = txn_sorted.groupby("account_id")["transaction_timestamp"].shift(-1)
txn_sorted["hours_to_next_debit"] = (
    txn_sorted["next_ts"] - txn_sorted["transaction_timestamp"]
).dt.total_seconds() / 3600

credit_to_debit = txn_sorted[
    (txn_sorted["txn_type"] == "C") & (txn_sorted["next_type"] == "D")
]

rapid_agg = credit_to_debit.groupby("account_id")["hours_to_next_debit"].agg(
    min_hours_credit_to_debit="min",
    median_hours_credit_to_debit="median"
).reset_index()

rapid_labeled = rapid_agg.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")
print("\n--- Min Hours from Credit to Next Debit ---")
print(rapid_labeled.groupby("is_mule")["min_hours_credit_to_debit"].describe().round(1))

rapid_labeled["ultra_rapid"] = (rapid_labeled["min_hours_credit_to_debit"] < 24).astype(int)
print("\n--- Ultra Rapid Pass-Through (< 24 hrs credit to debit) ---")
print(pd.crosstab(rapid_labeled["ultra_rapid"], rapid_labeled["is_mule"], normalize="columns").round(3))


RAPID PASS-THROUGH ANALYSIS

--- Min Hours from Credit to Next Debit ---
           count    mean     std  min  25%   50%    75%      max
is_mule                                                         
0        13792.0  1255.4  4014.1  0.0  2.3  27.5  210.8  40971.6
1          187.0  1103.1  3616.6  0.0  1.8   8.1  111.5  30703.2

--- Ultra Rapid Pass-Through (< 24 hrs credit to debit) ---
is_mule          0      1
ultra_rapid              
0            0.524  0.422
1            0.476  0.578


# COUNTERPARTY NETWORK ANALYSIS

In [ ]:
# Shared counterparties between mule accounts
mule_ids = labels[labels.is_mule == 1]["account_id"].tolist()
legit_ids = labels[labels.is_mule == 0]["account_id"].tolist()

mule_txns = transactions[transactions["account_id"].isin(mule_ids)]
mule_counterparties = set(mule_txns["counterparty_id"].dropna().unique())
print(f"Unique counterparties of mule accounts: {len(mule_counterparties):,}")

legit_txns = transactions[transactions["account_id"].isin(legit_ids)]
legit_counterparties = set(legit_txns["counterparty_id"].dropna().unique())
print(f"Unique counterparties of legit accounts: {len(legit_counterparties):,}")

shared = mule_counterparties & legit_counterparties
print(f"Counterparties shared between mule and legit: {len(shared):,}")
print(f"Mule counterparties exclusive to mules: {len(mule_counterparties - legit_counterparties):,}")

# Accounts that transact with known mules
all_txns = transactions.copy()
mule_as_counterparty = all_txns[all_txns["counterparty_id"].isin(mule_ids)]
accounts_touching_mules = mule_as_counterparty["account_id"].unique()
print(f"\nAccounts that transact with a known mule account: {len(accounts_touching_mules):,}")

# How many of those are in train labels?
touching_labeled = labels[labels["account_id"].isin(accounts_touching_mules)]
print(f"  Of those, labeled: {len(touching_labeled)}")
print(f"  Of those labeled, are mule: {touching_labeled['is_mule'].sum()}")
print(f"  Mule rate among accounts touching mules: {touching_labeled['is_mule'].mean():.4f}")



COUNTERPARTY NETWORK ANALYSIS
Unique counterparties of mule accounts: 2,377
Unique counterparties of legit accounts: 77,612
Counterparties shared between mule and legit: 1,790
Mule counterparties exclusive to mules: 587

Accounts that transact with a known mule account: 0
  Of those, labeled: 0
  Of those labeled, are mule: 0
  Mule rate among accounts touching mules: nan


# POST-MOBILE-UPDATE SPIKE

In [ ]:
accts_with_mobile_update = accounts[accounts["last_mobile_update_date"].notna()][
    ["account_id", "last_mobile_update_date"]
]

# Merging with transactions
mobile_txns = transactions.merge(accts_with_mobile_update, on="account_id", how="inner")
mobile_txns["days_since_update"] = (
    mobile_txns["transaction_timestamp"] - mobile_txns["last_mobile_update_date"]
).dt.days

# Counting txns in 30 days before and after update
before_30 = mobile_txns[mobile_txns["days_since_update"].between(-30, 0)].groupby("account_id")["amount"].count().rename("txn_count_before_30d")
after_30  = mobile_txns[mobile_txns["days_since_update"].between(0, 30)].groupby("account_id")["amount"].count().rename("txn_count_after_30d")

mobile_spike = pd.concat([before_30, after_30], axis=1).fillna(0).reset_index()
mobile_spike["spike_ratio"] = (mobile_spike["txn_count_after_30d"] + 1) / (mobile_spike["txn_count_before_30d"] + 1)
mobile_spike = mobile_spike.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")

print("\n--- Transaction Spike Ratio after Mobile Update (after_30d / before_30d) ---")
print(mobile_spike.groupby("is_mule")["spike_ratio"].describe().round(2))
print("\n--- Accounts with spike_ratio > 3 ---")
print(pd.crosstab((mobile_spike["spike_ratio"] > 3).astype(int), mobile_spike["is_mule"], normalize="columns").round(3))



POST-MOBILE-UPDATE SPIKE

--- Transaction Spike Ratio after Mobile Update (after_30d / before_30d) ---
         count   mean    std   min   25%   50%  75%    max
is_mule                                                   
0        884.0   3.57  10.62  0.01  0.33  1.00  3.0  146.0
1         16.0  17.60  34.63  0.04  0.46  1.88  5.0  111.0

--- Accounts with spike_ratio > 3 ---
is_mule          0      1
spike_ratio              
0            0.798  0.625
1            0.202  0.375


# FAN-IN / FAN-OUT ANALYSIS

In [ ]:
credit_counterparty_count = (credits.groupby("account_id")["counterparty_id"]
                              .nunique().rename("unique_credit_sources").reset_index())
debit_counterparty_count  = (debits.groupby("account_id")["counterparty_id"]
                              .nunique().rename("unique_debit_targets").reset_index())

fan = credit_counterparty_count.merge(debit_counterparty_count, on="account_id", how="outer").fillna(0)
fan = fan.merge(labels[["account_id", "is_mule"]], on="account_id", how="inner")

print("\n--- Unique Credit Sources (Fan-In) ---")
print(fan.groupby("is_mule")["unique_credit_sources"].describe().round(1))

print("\n--- Unique Debit Targets (Fan-Out) ---")
print(fan.groupby("is_mule")["unique_debit_targets"].describe().round(1))

# High fan-in: many sources, few debit targets
fan["fan_in_indicator"]  = (fan["unique_credit_sources"] > fan["unique_credit_sources"].quantile(0.9)).astype(int)
fan["fan_out_indicator"] = (fan["unique_debit_targets"]  > fan["unique_debit_targets"].quantile(0.9)).astype(int)
print("\n--- Fan-In Indicator (top 10% credit sources) ---")
print(pd.crosstab(fan["fan_in_indicator"],  fan["is_mule"], normalize="columns").round(3))
print("\n--- Fan-Out Indicator (top 10% debit targets) ---")
print(pd.crosstab(fan["fan_out_indicator"], fan["is_mule"], normalize="columns").round(3))



FAN-IN / FAN-OUT ANALYSIS

--- Unique Credit Sources (Fan-In) ---
           count  mean  std  min  25%  50%  75%   max
is_mule                                              
0        19571.0   5.0  5.6  0.0  1.0  3.0  7.0  40.0
1          227.0   6.8  7.8  0.0  2.0  4.0  8.5  60.0

--- Unique Debit Targets (Fan-Out) ---
           count  mean  std  min  25%  50%  75%   max
is_mule                                              
0        19571.0   5.5  5.9  0.0  1.0  3.0  8.0  44.0
1          227.0   7.2  7.8  0.0  2.0  4.0  9.0  62.0

--- Fan-In Indicator (top 10% credit sources) ---
is_mule               0      1
fan_in_indicator              
0                 0.909  0.863
1                 0.091  0.137

--- Fan-Out Indicator (top 10% debit targets) ---
is_mule                0      1
fan_out_indicator              
0                  0.906  0.863
1                  0.094  0.137


# INCOME MISMATCH ANALYSIS

In [ ]:
income_mismatch = (txn_labeled[["account_id", "max_credit_amount", "is_mule"]]
                   .merge(accounts[["account_id", "avg_balance"]], on="account_id", how="left"))

income_mismatch["max_credit_to_balance"] = (
    income_mismatch["max_credit_amount"] / (income_mismatch["avg_balance"].abs() + 1)
)

print("\n--- Max Single Credit / Avg Balance Ratio ---")
print(income_mismatch.groupby("is_mule")["max_credit_to_balance"].describe().round(2))

high_mismatch = (income_mismatch["max_credit_to_balance"] > 10).astype(int)
print("\n--- High Mismatch (max_credit > 10x avg_balance) ---")
print(pd.crosstab(high_mismatch, income_mismatch["is_mule"], normalize="columns").round(3))


INCOME MISMATCH ANALYSIS

--- Max Single Credit / Avg Balance Ratio ---
           count     mean       std    min   25%   50%    75%        max
is_mule                                                                 
0        16594.0  4706.69  47204.73 -41.15  0.15  1.52  15.37  3191000.0
1          200.0  5663.65  29072.58   0.00  0.75  6.53  44.09   261000.0

--- High Mismatch (max_credit > 10x avg_balance) ---
is_mule                    0      1
max_credit_to_balance              
0                      0.754  0.617
1                      0.246  0.383


# DATA QUALITY SUMMARY

In [ ]:
print("\n" + "=" * 60)
print("")
print("=" * 60)

print(f"Mules with flag before account open:   {(mules['days_open_to_flag'] < 0).sum()}")
print(f"Mules with no alert reason:            {labels[labels.is_mule==1]['alert_reason'].isna().sum()}")
print(f"Mules flagged as 'Routine Investigation': {(labels[labels.is_mule==1]['alert_reason']=='Routine Investigation').sum()}")
print(f"Mule flag dates in 2026 (beyond window): {(labels[labels.is_mule==1]['mule_flag_date'].dt.year == 2026).sum()}")
print(f"Accounts in train+test vs total accounts: {len(labels)+len(test)} vs {len(accounts)}")

# Duplicate check
print(f"\nDuplicate account_ids in labels: {labels['account_id'].duplicated().sum()}")
print(f"Duplicate account_ids in accounts: {accounts['account_id'].duplicated().sum()}")
print(f"Duplicate transaction_ids: {transactions['transaction_id'].duplicated().sum()}")

# Overlap check
train_ids = set(labels["account_id"])
test_ids  = set(test["account_id"])
overlap = train_ids & test_ids
print(f"\nTrain/test account overlap: {len(overlap)} (should be 0)")

print("\n" + "=" * 60)
print("EDA COMPLETE")
print("=" * 60)


DATA QUALITY SUMMARY
Mules with flag before account open:   27
Mules with no alert reason:            21
Mules flagged as 'Routine Investigation': 55
Mule flag dates in 2026 (beyond window): 11
Accounts in train+test vs total accounts: 40038 vs 40038

Duplicate account_ids in labels: 0
Duplicate account_ids in accounts: 0
Duplicate transaction_ids: 0

Train/test account overlap: 0 (should be 0)

EDA COMPLETE
